# Evaluation of model performance

Evaluation...  

...  

...  


...

In [20]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
from torch.utils.data import DataLoader
import scipy.stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split,KFold

from src.data.data_scaler import PreProcess
from src.data.data_loader import MIMICDataset,import_data,collate_fn_padd
from src.utils import setup_logger
from src.training.training_nn import *
from src.models.models import ODERNN,LatentODE1
from src.utils import seed_everything

In [32]:
import scipy

In [39]:
(1-alpha/2)

0.975

In [40]:
alpha = 0.05
scipy.stats.norm.ppf(1-alpha/2)

1.959963984540054

In [2]:
import plotnine as p9

In [3]:
from data.feature_sets import all_features

In [4]:
def glc_transform(x):
    x = x.copy()
    x[x > 0] = np.log(x[x > 0]) - np.log(140)
    return x

def glc_invtransform(x):
    x = x.copy()
    x = np.exp(x + np.log(140))
    return x

ginv = glc_invtransform

In [23]:
df = import_data('data/analysis.csv')
df.sort_values(by=['icustay_id','timer'],inplace=True)
df.reset_index(drop=True,inplace=True)
ids = df.icustay_id.unique()
NFOLDS = 5
FEATURES = all_features()
BATCH_SIZE = 128
kf = KFold(n_splits=NFOLDS)
kf.get_n_splits(ids)
for fold, (train_index, test_index) in enumerate(kf.split(ids)):
    if fold == 0:

        # train-test split
        trains_ids,test_ids = ids[train_index],ids[test_index]
        df_train = df.loc[df.icustay_id.isin(trains_ids)].copy(deep=True)
        df_test = df.loc[df.icustay_id.isin(test_ids)].copy(deep=True)
        print(df_test.shape)

        # train-valid split
        train_ids, valid_ids = train_test_split(df_train.icustay_id.unique(),test_size=0.1)
        df_valid = df_train.loc[df_train.icustay_id.isin(valid_ids)].copy(deep=True)
        df_train = df_train.loc[df_train.icustay_id.isin(train_ids)].copy(deep=True)

        # preprocess
        preproc = PreProcess(FEATURES,QuantileTransformer())
        preproc.fit(df_train)
        df_train = preproc.transform(df_train)
        df_valid = preproc.transform(df_valid)
        df_test = preproc.transform(df_test)
        dl_train = DataLoader(MIMICDataset(df_train,FEATURES,pad=SEQUENCE_LEN),
                          batch_size=BATCH_SIZE,collate_fn=collate_fn_padd)
        dl_valid = DataLoader(MIMICDataset(df_valid,FEATURES,pad=SEQUENCE_LEN),
                          batch_size=BATCH_SIZE,collate_fn=collate_fn_padd)
        dl_test = DataLoader(MIMICDataset(df_test,FEATURES,pad=SEQUENCE_LEN),
                          batch_size=BATCH_SIZE,collate_fn=collate_fn_padd)
        dataloaders = {'train':dl_train,'validation':dl_valid,'test':dl_test}


excluding: 200024
excluding: 200586
excluding: 201104
excluding: 201173
excluding: 201605
excluding: 203383
excluding: 205463
excluding: 205830
excluding: 206288
excluding: 206986
excluding: 208691
excluding: 208937
excluding: 212690
excluding: 212925
excluding: 214617
excluding: 215027
excluding: 215250
excluding: 216192
excluding: 216504
excluding: 216867
excluding: 217645
excluding: 218713
excluding: 218874
excluding: 219153
excluding: 219305
excluding: 220024
excluding: 220671
excluding: 220704
excluding: 221494
excluding: 221514
excluding: 221574
excluding: 221812
excluding: 222096
excluding: 223717
excluding: 224428
excluding: 226283
excluding: 227040
excluding: 227991
excluding: 228262
excluding: 229133
excluding: 229142
excluding: 229929
excluding: 230012
excluding: 230384
excluding: 231775
excluding: 232042
excluding: 232582
excluding: 233553
excluding: 234981
excluding: 235824
excluding: 236999
excluding: 237956
excluding: 238662
excluding: 240029
excluding: 240043
excluding:

NameError: name 'FEATURES' is not defined

In [5]:
preproc = PreProcess(all_features(),QuantileTransformer())
df_train = pd.read_csv('data/train.csv')
preproc.fit(df_train)

## Import model

In [6]:
model_path = 'experiments/mimic/test.pth'
cpt = torch.load(model_path)

In [7]:
model = ODERNN(99, 12, 0.1, 1,128,"cpu").to("cpu")

In [8]:
model.load_state_dict(cpt['state_dict'])

<All keys matched successfully>

In [9]:
model

ODERNN(
  (rnn): RNNCell(99, 12)
  (func): ODERNN_(
    (net): Sequential(
      (0): Linear(in_features=12, out_features=50, bias=True)
      (1): Tanh()
      (2): Linear(in_features=50, out_features=12, bias=True)
    )
  )
  (relu): ReLU()
  (l1): Linear(in_features=12, out_features=6, bias=True)
  (distribution_mu): Linear(in_features=6, out_features=1, bias=True)
  (sigma_net): Sequential(
    (0): Linear(in_features=12, out_features=12, bias=True)
    (1): Tanh()
    (2): Linear(in_features=12, out_features=1, bias=True)
    (3): Softplus(beta=1, threshold=20)
  )
)

## Import test data

In [10]:
df_test = import_data("data/test.csv")
print(df_test.shape)

(78557, 112)


In [11]:
df_test = preproc.transform(df_test)

In [12]:
df_test['dt'] = df_test.timer_dt - df_test.timer

## Check

In [13]:
tmp = MIMICDataset(df_test,all_features(),verbose=False)

In [14]:
glc_transform(np.array([198.,189.]))

array([0.34662461, 0.30010459])

In [15]:
x, y, msk, dt, msk0 = tmp[0]
print(ginv(x[0,0]))
x[0,1] = 0.0
for dt_ in np.linspace(0,3,10):
    dt[0,1] = dt[0,0] + dt_
    print(ginv(model.forward(torch.tensor(dt).unsqueeze(0),torch.tensor(x).unsqueeze(0))[0][0,0].detach().numpy()))

198.00000093515325
[146.3276]
[146.25575]
[146.17912]
[146.10303]
[146.02745]
[145.95235]
[145.87782]
[145.80383]
[145.73042]
[145.65756]


# Predict

In [16]:
dl_test = DataLoader(MIMICDataset(df_test,all_features()),batch_size=256,collate_fn=collate_fn_padd)
dataloaders = {'test':dl_test}
loss_test,error_tests,y_preds,y_tests,msks = model.evaluate(dataloaders["test"])

reconfiguring data...
excluded patients: 0
excluded rows: 0


100%|██████████| 17/17 [00:28<00:00,  1.67s/it]

_rmse : 55.492
_loss : 0.297


In [24]:
y_obs = df_test.loc[df_test.msk==0,'glc_dt'].to_numpy()

In [25]:
y_obs.shape[0]

60624

In [17]:
mu,sig = model.predict(dataloaders["test"])
print(mu.shape)
print(sig.shape)

(71414,)
(71414,)


In [26]:
mu_preds = []

In [18]:
print(df_test.loc[df_test.msk==0].shape)

(71414, 113)


In [ ]:
plt.hist(mu)

## Evaluate

text

In [ ]:
df_test.loc[df_test.msk==0,'mu'] = mu
df_test.loc[df_test.msk==0,'sig'] = sig

In [ ]:
df_test['lower80'] = df_test['mu'] - 1.281552*df_test['sig']
df_test['upper80'] = df_test['mu'] + 1.281552*df_test['sig']
df_test['lower95'] = df_test['mu'] - 1.959964*df_test['sig']
df_test['upper95'] = df_test['mu'] + 1.959964*df_test['sig']
df_test['lower99'] = df_test['mu'] - 3.290527*df_test['sig']
df_test['upper99'] = df_test['mu'] + 3.290527*df_test['sig']

In [ ]:
(
    p9.ggplot(df_test.loc[df_test.msk==0],p9.aes(x='ginv(mu)')) + 
    p9.geom_histogram(bins=100) + 
    p9.coord_cartesian(xlim=(0,500)) +
    p9.labs(x="Predicted blood glucose (mg/dL)") +
    p9.theme_bw(base_size=14)
)

In [ ]:
(
    p9.ggplot(df_test.loc[df_test.msk==0],p9.aes(x='ginv(glc_dt)')) + 
    p9.geom_histogram(bins=100) + 
    p9.labs(x="Observed blood glucose (mg/dL)") +
    p9.theme_bw(base_size=14)
)

In [ ]:
(
    p9.ggplot(df_test.loc[df_test.msk==0],p9.aes(x='ginv(mu)',y='ginv(glc_dt)')) + 
    p9.geom_point(alpha=0.5) + 
    p9.geom_smooth(color="blue") +
    p9.geom_abline(intercept=0,slope=1,linetype="dashed",color="red") +
    p9.coord_cartesian(xlim=(0,500),ylim=(0,500)) +
    p9.labs(x="Observed blood glucose (mg/dL)",y= "Predicted blood glucose (mg/dL)") +
    p9.theme_bw()
)

In [ ]:
res80 = sum((df_test['lower80'] < df_test.glc_dt) & (df_test['upper80'] > df_test.glc_dt) & (df_test.msk == 0)) / sum(df_test.msk == 0)
res95 = sum((df_test['lower95'] < df_test.glc_dt) & (df_test['upper95'] > df_test.glc_dt) & (df_test.msk == 0)) / sum(df_test.msk == 0)
res99 = sum((df_test['lower99'] < df_test.glc_dt) & (df_test['upper99'] > df_test.glc_dt) & (df_test.msk == 0)) / sum(df_test.msk == 0)
print("coverage of 80% prediction interval: {:05.4f}".format(res80))
print("coverage of 95% prediction interval: {:05.4f}".format(res95))
print("coverage of 99% prediction interval: {:05.4f}".format(res99))

In [ ]:
N = 12
max_t = 12
conf_res = pd.DataFrame({
    "dt0" : np.linspace(0,max_t-1,N),
    "dt1" : np.linspace(1,max_t,N),
    "cov80" : np.zeros(N),
    "cov95" : np.zeros(N),
    "cov99" : np.zeros(N)
})
for i in range(conf_res.shape[0]):
    msk = (df_test.msk==0) & (df_test.dt >= conf_res.dt0[i]) & (df_test.dt < conf_res.dt1[i])
    res80 = sum((df_test['lower80'] < df_test.glc_dt) & (df_test['upper80'] > df_test.glc_dt) & msk) / sum(msk)
    res95 = sum((df_test['lower95'] < df_test.glc_dt) & (df_test['upper95'] > df_test.glc_dt) & msk) / sum(msk)
    res99 = sum((df_test['lower99'] < df_test.glc_dt) & (df_test['upper99'] > df_test.glc_dt) & msk) / sum(msk)
    conf_res.cov80[i] = res80
    conf_res.cov95[i] = res95
    conf_res.cov99[i] = res99

In [ ]:
(
    p9.ggplot(conf_res,p9.aes(x="dt0",y="cov95")) +
    p9.geom_point() +
    p9.geom_hline(yintercept=0.95,linetype="dashed",color="red") +
    p9.labs(x="Forecast time ahead (hours)",
             y="Coverage of\n95% prediction interval",
             title="all (to crop)") +
    p9.theme_bw() +
    p9.coord_cartesian(ylim=(0.9,1.0)) +
    p9.scale_y_continuous(breaks=np.linspace(0.9,1.0,5),labels=np.linspace(0.9,1.0,5)*100) +
    p9.scale_x_continuous(breaks=np.linspace(0,12,5),labels=["0-1","3-4","6-7","9-10","12-13"]) 
)

In [ ]:
N = 12
max_t = 12
conf_res_hypo = pd.DataFrame({
    "dt0" : np.linspace(0,max_t-1,N),
    "dt1" : np.linspace(1,max_t,N),
    "cov80" : np.zeros(N),
    "cov95" : np.zeros(N),
    "cov99" : np.zeros(N)
})
for i in range(conf_res_hypo.shape[0]):
    msk = (df_test.msk==0) & (df_test.dt >= conf_res_hypo.dt0[i]) & (df_test.dt < conf_res_hypo.dt1[i]) & (ginv(df_test.glc_dt) < 70)
    res80 = sum((df_test['lower80'] < df_test.glc_dt) & (df_test['upper80'] > df_test.glc_dt) & msk) / sum(msk)
    res95 = sum((df_test['lower95'] < df_test.glc_dt) & (df_test['upper95'] > df_test.glc_dt) & msk) / sum(msk)
    res99 = sum((df_test['lower99'] < df_test.glc_dt) & (df_test['upper99'] > df_test.glc_dt) & msk) / sum(msk)
    conf_res_hypo.cov80[i] = res80
    conf_res_hypo.cov95[i] = res95
    conf_res_hypo.cov99[i] = res99

In [ ]:
(
    p9.ggplot(conf_res_hypo,p9.aes(x="dt0",y="cov95")) +
    p9.geom_point() +
    p9.geom_hline(yintercept=0.95,linetype="dashed",color="red") +
    p9.labs(x="Forecast time ahead (hours)",
             y="Coverage of\n95% prediction interval",
             title="hypo (to crop)") +
    p9.theme_bw() +
    p9.coord_cartesian(ylim=(0.0,1.0)) +
    #plt9.scale_y_continuous(breaks=np.linspace(0.9,1.0,5),labels=np.linspace(0.9,1.0,5)*100) +
    p9.scale_x_continuous(breaks=np.linspace(0,12,5),labels=["0-1","3-4","6-7","9-10","12-13"]) 
)

In [ ]:
N = 12
max_t = 12
conf_res_hyper = pd.DataFrame({
    "dt0" : np.linspace(0,max_t-1,N),
    "dt1" : np.linspace(1,max_t,N),
    "cov80" : np.zeros(N),
    "cov95" : np.zeros(N),
    "cov99" : np.zeros(N)
})
for i in range(conf_res_hyper.shape[0]):
    msk = (df_test.msk==0) & (df_test.dt >= conf_res_hyper.dt0[i]) & (df_test.dt < conf_res_hyper.dt1[i]) & (ginv(df_test.glc_dt) > 180)
    res80 = sum((df_test['lower80'] < df_test.glc_dt) & (df_test['upper80'] > df_test.glc_dt) & msk) / sum(msk)
    res95 = sum((df_test['lower95'] < df_test.glc_dt) & (df_test['upper95'] > df_test.glc_dt) & msk) / sum(msk)
    res99 = sum((df_test['lower99'] < df_test.glc_dt) & (df_test['upper99'] > df_test.glc_dt) & msk) / sum(msk)
    conf_res_hyper.cov80[i] = res80
    conf_res_hyper.cov95[i] = res95
    conf_res_hyper.cov99[i] = res99

In [ ]:
(
    p9.ggplot(conf_res_hyper,p9.aes(x="dt0",y="cov95")) +
    p9.geom_point() +
    p9.geom_hline(yintercept=0.95,linetype="dashed",color="red") +
    p9.labs(x="Forecast time ahead (hours)",
             y="Coverage of\n95% prediction interval",
             title="hyper (to crop)") +
    p9.theme_bw() +
    p9.coord_cartesian(ylim=(0.75,1.0)) +
    #plt9.scale_y_continuous(breaks=np.linspace(0.9,1.0,5),labels=np.linspace(0.9,1.0,5)*100) +
    p9.scale_x_continuous(breaks=np.linspace(0,12,5),labels=["0-1","3-4","6-7","9-10","12-13"]) 
)

In [ ]:
df_test["glc_dt_locf"] = df_test.glc
df_test.loc[df_test.glc_dt_locf == 0,'glc_dt_locf'] = np.NaN
df_test["glc_dt_locf"] = df_test.groupby('icustay_id')["glc_dt_locf"].ffill()
res_rmse_locf = math.sqrt(np.mean((ginv(df_test.glc_dt_locf[df_test.msk==0]) - ginv(df_test.glc_dt[df_test.msk==0]))**2))
print("RMSE (locf): {:05.4f}".format(res_rmse_locf))

In [ ]:
res_rmse_model = math.sqrt(np.mean((ginv(df_test.mu[df_test.msk==0]) - ginv(df_test.glc_dt[df_test.msk==0]))**2))
print("RMSE (model): {:05.4f}".format(res_rmse_model))

In [ ]:
N = 12
max_t = 12
rmse_res = pd.DataFrame({
    "dt0" : np.linspace(0,max_t-1,N),
    "dt1" : np.linspace(1,max_t,N),
    "odernn" : np.zeros(N),
    "locf" : np.zeros(N)
})
for i in range(rmse_res.shape[0]):
    msk = (df_test.msk==0) & (df_test.dt >= rmse_res.dt0[i]) & (df_test.dt < rmse_res.dt1[i]) 
    res_rmse_locf = math.sqrt(np.mean((ginv(df_test.glc_dt_locf[msk]) - ginv(df_test.glc_dt[msk]))**2))
    res_rmse_odernn = math.sqrt(np.mean((ginv(df_test.mu[msk]) - ginv(df_test.glc_dt[msk]))**2))
    rmse_res.odernn[i] = res_rmse_odernn
    rmse_res.locf[i] = res_rmse_locf

In [ ]:
rmse_res.index = pd.MultiIndex.from_arrays(rmse_res[['dt0', 'dt1']].values.T, names=['dt0', 'dt1'])

In [ ]:
rmse_res = rmse_res.loc[:,["odernn","locf"]].stack()

In [ ]:
rmse_res = pd.concat((rmse_res.index.to_frame().reset_index(drop=True),rmse_res.reset_index(drop=True)),axis=1)

In [ ]:
rmse_res = rmse_res.rename(columns={2:"model",0:"rmse"})

In [ ]:
(
    p9.ggplot(rmse_res,p9.aes(x="dt0",y="rmse",color="factor(model)")) +
    p9.geom_point() +
    p9.geom_line() +
    p9.labs(x="Forecast time ahead (hours)",
             y="RMSE",
             title="all (to crop)") +
    p9.theme_bw() +
    p9.scale_color_discrete(name=" ") +
    #plt9.coord_cartesian(ylim=(0.75,1.0)) +
    #plt9.scale_y_continuous(breaks=np.linspace(0.9,1.0,5),labels=np.linspace(0.9,1.0,5)*100) +
    p9.scale_x_continuous(breaks=np.linspace(0,12,5),labels=["0-1","3-4","6-7","9-10","12-13"]) 
)

In [ ]:
prob_test = scipy.stats.norm(df_test.loc[df_test.msk==0,"mu"], df_test.loc[df_test.msk==0,"sig"]).cdf(df_test.loc[df_test.msk==0,"glc_dt"])

In [ ]:
# a forecast is overdispersed if var < 0.8333 and underdispersed if var > 0.8333
print(1/12)
varF = np.var(prob_test)
print('variance of probs: {:05.4f}'.format(varF))
if varF > 1/12:
    print("underdispersed")
else: 
    print("overdispersed")

In [ ]:
(
    p9.ggplot(pd.DataFrame(prob_test),p9.aes(x="prob_test")) +
    p9.geom_histogram(bins=45,color="white",fill="lightblue") + 
    p9.labs(x="Probability integral transform") +
    p9.scale_x_continuous(breaks=np.linspace(0,1,6)) +
    p9.theme_bw(base_size=14)
)

In [ ]:
# quantile plot
def calibration_plot(prob_test):
    nbins = 20
    x1s = plt.hist(prob_test,bins=nbins)
    plt.close()
    xbin = np.zeros(nbins)
    for i in range(nbins):
        xbin[i] = (x1s[1][i] + x1s[1][i+1])/2
    qs = np.cumsum(x1s[0])/sum(x1s[0])
    errors = np.sqrt(qs*(1-qs)/sum(x1s[0]))
    df = pd.DataFrame({"predq":xbin,"empq":qs,"error":errors})
    p = (
        p9.ggplot(df,p9.aes(x="predq",y="empq")) +
        p9.geom_point() + 
        p9.theme_bw(base_size=14) +
        p9.labs(x="Predicted quantile",y="Empirical quantile") +
        p9.geom_abline(intercept=0,slope=1,linetype="dashed",color="red")
    )
    fig = p.draw()
    fig.show()
calibration_plot(prob_test)

In [ ]:
df_test_input = df_test.loc[df_test.input_hrs > 0].copy()
samp_ids = np.random.choice(df_test_input.icustay_id.unique(),min(100,df_test_input.icustay_id.unique().shape[0]))
df_test_input = df_test_input[df_test_input.icustay_id.isin(samp_ids)]

In [ ]:
inputs = np.linspace(0.7,1.0,3)
inputs = np.append(0.0,inputs)
dts = np.linspace(0.0,2.0,5)
for i in range(0,inputs.shape[0]):
    insulin_input = inputs[i]
    df_test_input["input_hrs"] = insulin_input
    for j in range(0,dts.shape[0]):
        df_test_input["timer_dt"] = df_test_input["timer"] + dts[j]
        dl_test_input = DataLoader(MIMICDataset(df_test_input,all_features(),verbose=False),batch_size=128,collate_fn=collate_fn_padd)
        dataloaders = {'test':dl_test_input}
        mu,sig = model.predict(dataloaders["test"])
        df_test_input.loc[df_test_input.msk==0,'mu'+str(i)+str(j)] = mu
        #df_test_input.loc[df_test_input.msk==0,'sig'+str(i)] = sig
        df_test_input.loc[df_test_input.msk==0,'change'+str(i)+str(j)] = ginv(df_test_input.loc[df_test_input.msk==0,'glc']) - ginv(mu)

In [ ]:
cols = [col for col in df_test_input if col.startswith('change')]
df_test_input = df_test_input.loc[df_test_input.msk == 0,cols]
nn = df_test_input.shape[0]
df_test_input = df_test_input.stack()

In [ ]:
df_test_input = pd.concat((df_test_input.index.to_frame().reset_index(drop=True),df_test_input.reset_index(drop=True)),axis=1)
df_test_input.columns = ["id","change","value"]

In [ ]:
inputsr = np.repeat(inputs,(nn,))
inputsr = np.tile(inputsr,(dts.shape[0],))
dtsr = np.tile(dts,(nn,))
dtsr = np.tile(dtsr,(inputs.shape[0],))
df_test_input['dt'] = dtsr
df_test_input['input'] = inputsr

In [ ]:
df_test_input

In [ ]:
(
    p9.ggplot(df_test_input,p9.aes(x="dt",y="value",color="factor(input)")) +
    #p9.geom_line() +
    p9.geom_smooth() +
    p9.coord_cartesian(ylim=(0,10)) +
    p9.theme_bw(base_size=14)
)

In [ ]:
df_test_input.loc[(df_test_input.input==0) & (df_test_input.id == 325)]

In [ ]:
df_test_input["id1"] = np.repeat(range(0,8984),5)

In [ ]:
(
    p9.ggplot(df_test_input.loc[df_test_input.input==0][0:200],p9.aes(x="dt",y="-value")) +
    p9.geom_line(p9.aes(group="id1"),alpha=0.2) +
    p9.theme_bw(base_size=14) +
    p9.geom_smooth(color="blue")+
    p9.facet_wrap("input")+
    p9.labs(x="Forecast time ahead (hours)",
            y="Forecast change\nin blood glucose (mg/dL)",
           title="no insulin")
)

In [ ]:
(
    p9.ggplot(df_test_input.loc[(df_test_input.input >= 0.7) & (df_test_input.input < 0.8)][0:200],p9.aes(x="dt",y="-value")) +
    p9.geom_line(p9.aes(group="id1"),alpha=0.2) +
    p9.theme_bw(base_size=14) +
    p9.geom_smooth(color="blue")+
    p9.facet_wrap("input")+
    p9.labs(x="Forecast time ahead (hours)",
            y="Forecast change\nin blood glucose (mg/dL)",
           title="0.75 std insulin")
)

In [ ]:
(
    p9.ggplot(df_test_input.loc[(df_test_input.input >= 0.85) & (df_test_input.input < 1.0)],p9.aes(x="dt",y="-value")) +
    p9.geom_line(p9.aes(group="id1"),alpha=0.2) +
    p9.theme_bw(base_size=14) +
    p9.geom_smooth(color="blue")+
    p9.facet_wrap("input")+
    p9.labs(x="Forecast time ahead (hours)",
            y="Forecast change\nin blood glucose (mg/dL)",
           title="0.85 std insulin")
)

In [ ]:
df_test['change_mgdl'] = ginv(df_test.loc[df_test.msk==0].mu) - ginv(df_test.loc[df_test.msk==0].glc)

In [ ]:
df_test['mu_inv'] = ginv(df_test['mu'])
df_test['glc_inv'] = ginv(df_test['glc'])
df_test['glc_dt_inv'] = ginv(df_test['glc_dt'])

In [ ]:
df_test.loc[df_test.icustay_id == 299957,["timer","timer_dt","glc_inv","mu_inv","glc_dt_inv","msk","input_hrs"]]

In [ ]:
(
    p9.ggplot(df_test.loc[df_test.input_hrs>0],p9.aes(x="input_hrs",y="change_mgdl"))+
    p9.geom_point(alpha=0.5) +
    p9.geom_smooth(color="blue") +
    p9.coord_cartesian(ylim=(-150,25))+
    p9.labs(x="Insulin infusion input (??)",y="Forecast change in\nblood glucose (mg/dL)") +
    p9.theme_bw(base_size=14)
)